In [ ]:
# default_exp core

# Iterative_masking

> Use MSA Transformer to generate synthetic protein sequences by masking iteratively the same MSA.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
def test():
    return 0

## Build library

In [ ]:
# hide

# RUN THIS CELL EVERYTIME YOU CHANGE THE NOTEBOOK SO THAT IT BUILDS THE NEW LIBRARY 
# -------->    no need to run nbdev_build_lib on the terminal
from nbdev.export import notebook2script
notebook2script()
# !nbdev_clean_nbs
!nbdev_build_docs

Converted 00_core.ipynb.
Converted index.ipynb.
converting: /home/damiano/Documents/Projects/Iterative_masking/00_core.ipynb
converting: /home/damiano/Documents/Projects/Iterative_masking/index.ipynb
converting /home/damiano/Documents/Projects/Iterative_masking/index.ipynb to README.md
